<a href="https://colab.research.google.com/github/Avani1994/Emjoi_Recognition/blob/main/TrainChatImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 216.3MB 74kB/s 
     |████████████████████████████████| 307kB 45.6MB/s 
     |████████████████████████████████| 3.1MB 48.3MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 890kB 50.2MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=3a9bc98a1875dcec92ad1c7329e40996592858666c3c1cb1382a42e21f39d1b5
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: tensorflow 2.2.0 has requirement keras-preprocessing>=1.1.0, but you'll have keras-preprocessing 1.0.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.8.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
 

In [ ]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI/lib64

In [ ]:
!unzip ChatImageDataTest.zip -d ChatImageDataTest

In [ ]:
!unzip ChatImageDataTrain.zip -d ChatImageDataTrain

In [ ]:
!git clone https://github.com/kbardool/keras-frcnn.git

Cloning into 'keras-frcnn'...
remote: Enumerating objects: 589, done.
remote: Total 589 (delta 0), reused 0 (delta 0), pack-reused 589
Receiving objects: 100% (589/589), 173.38 KiB | 332.00 KiB/s, done.
Resolving deltas: 100% (402/402), done.


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# importing required libraries

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
from ast import literal_eval

# read the csv file using read_csv function of pandas
train = pd.read_csv("outputTrain.csv")
test = pd.read_csv("outputTest.csv")
train.head()

,id,boxes,char
0,image0,"[454, 164, 478, 188]",😝
1,image0,"[479, 164, 503, 188]",😦
2,image0,"[568, 178, 592, 193]",NaN
3,image0,"[221, 213, 245, 237]",😬
4,image0,"[246, 213, 270, 237]",☹


In [ ]:
train['char'] = train['char'].fillna("bluetick")
train['id'] = 'ChatImageDataTrain/ChatImageDataTrain/'+train['id']+'.png'
train['xmin'] = train['boxes'].apply(lambda x:literal_eval(x)[0])
train['ymin'] = train['boxes'].apply(lambda x:literal_eval(x)[1])
train['xmax'] = train['boxes'].apply(lambda x:literal_eval(x)[2])
train['ymax'] = train['boxes'].apply(lambda x:literal_eval(x)[3])
train.drop(['boxes'],axis=1,inplace=True)

In [ ]:
test['char'] = test['char'].fillna("bluetick")
test['id'] = 'ChatImageDataTest/ChatImageDataTest/'+test['id']+'.png'
test['xmin'] = test['boxes'].apply(lambda x:literal_eval(x)[0])
test['ymin'] = test['boxes'].apply(lambda x:literal_eval(x)[1])
test['xmax'] = test['boxes'].apply(lambda x:literal_eval(x)[2])
test['ymax'] = test['boxes'].apply(lambda x:literal_eval(x)[3])
test.drop(['boxes'],axis=1,inplace=True)

In [ ]:
train.head()

,id,char,xmin,ymin,xmax,ymax
0,ChatImageDataTrain/ChatImageDataTrain/image0.png,😝,454,164,478,188
1,ChatImageDataTrain/ChatImageDataTrain/image0.png,😦,479,164,503,188
2,ChatImageDataTrain/ChatImageDataTrain/image0.png,bluetick,568,178,592,193
3,ChatImageDataTrain/ChatImageDataTrain/image0.png,😬,221,213,245,237
4,ChatImageDataTrain/ChatImageDataTrain/image0.png,☹,246,213,270,237


In [ ]:
test.head()

,id,char,xmin,ymin,xmax,ymax
0,ChatImageDataTest/ChatImageDataTest/image1301.png,😒,463,146,487,170
1,ChatImageDataTest/ChatImageDataTest/image1301.png,😊,488,146,512,170
2,ChatImageDataTest/ChatImageDataTest/image1301.png,🤕,513,146,537,170
3,ChatImageDataTest/ChatImageDataTest/image1301.png,😟,538,146,562,170
4,ChatImageDataTest/ChatImageDataTest/image1301.png,bluetick,568,160,592,175


In [ ]:
data = pd.DataFrame()
data['format'] = train['id']

# as the images are in train_images folder, add train_images before the image name
#for i in range(data.shape[0]):
#    data['format'][i] = 'train_images/' + data['format'][i]

# add xmin, ymin, xmax, ymax and class as per the format required
for i in range(data.shape[0]):
    data['format'][i] = data['format'][i] + ',' + str(train['xmin'][i]) + ',' + str(train['ymin'][i]) + ',' + str(train['xmax'][i]) + ',' + str(train['ymax'][i]) + ',' + train['char'][i]

data.to_csv('annotate.txt', header=None, index=None, sep=' ')

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:
!python keras-frcnn/train_frcnn.py -o simple --num_epochs 500 -p annotate.txt

Using TensorFlow backend.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call la

In [ ]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.1.243


In [ ]:
!grep CUDNN_MAJOR -A 2 /usr/local/cuda/include/cudnn.h

grep: /usr/local/cuda/include/cudnn.h: No such file or directory
